## Import Libs & Fetch Dataset

This data is from [Ember](https://ember-energy.org/) which is an open data platform for worldwide energy data.

In [252]:
import os
import sys
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

In [253]:
PROJECT_ROOT = Path.cwd().parents[0]
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from my_project.paths import get_paths

paths = get_paths(PROJECT_ROOT)
DATA_DIR = paths['DATA_DIR']
RAW_DATA_DIR = paths['RAW_DATA_DIR']
PROCESSED_DATA_DIR = paths['PROCESSED_DATA_DIR']
LOGS_DIR = paths['LOGS_DIR']

from data.ember_api_client import EmberAPI
from utils.plots import save_static_img
from data.get_data import download_from_url

In [254]:
# get data from Ember API 
api = EmberAPI()

# get yearly generation data
params = {"is_aggregate_series":False, "start_date":"2000",}
yearly_generation_json = api.fetch_and_cache(
    endpoint_name="electricity_generation_yearly",
    fetch_func=api.electricity_generation_yearly,
    params=params
)

# at this time, the data returned from monthly generation endpoint is not complete
# compared to yearly generation endpoint, so we will use only the yearly data for now 
# get monthly generation data
# params = {"is_aggregate_series":False, "start_date":"2000-01", }
# monthly_generation_json = api.fetch_and_cache(
#     endpoint_name="electricity_generation_monthly",
#     fetch_func=api.electricity_generation_monthly,
#     params=params
# )

# there are no yearly capacity endpoint, so we will use monthly capacity data to 
# derive yearly capacity data by aggregating the monthly data
# get monthly installed capacity data
params = {"is_aggregate_series":False, "start_date": "2000-01",}
monthly_capacity_json = api.fetch_and_cache(
    endpoint_name = "electricity_capacity_monthly",
    fetch_func=api.electricity_capacity_monthly,
    params=params
)

2026-01-04 23:15:52 | INFO | Initialized EmberAPI client with base URL: https://api.ember-energy.org
2026-01-04 23:15:52 | INFO | Loading cached data from /home/zephyr/workspace/Global_Energy_Trends/data/raw/electricity_generation_yearly_is_aggregate_series-False_start_date-2000.json
2026-01-04 23:15:53 | INFO | Loading cached data from /home/zephyr/workspace/Global_Energy_Trends/data/raw/electricity_capacity_monthly_is_aggregate_series-False_start_date-2000-01.json


Decided to use yearly instead of monthly generation data since the yearly data is more comprehensive in terms of  coverage. As of Dec2025, the monthly generation data only contains 88 economies/countries whereas the yearly data contains over 210 economies/countries.

As for capacity data, only monthly data is available on the API, we will transform it later to match with generation data.

In [255]:
# df_generation_monthly = pd.DataFrame(monthly_generation_json.get("data",[]))
df_capacity_monthly = pd.DataFrame(monthly_capacity_json.get("data",[]))
df_generation_yearly = pd.DataFrame(yearly_generation_json.get("data", []))

In [256]:
df_capacity_monthly.sample(3)

,entity,entity_code,is_aggregate_entity,date,series,is_aggregate_series,capacity_gw,capacity_w_per_capita
1800,Belgium,BEL,False,2019-01-01,Wind offshore,False,1.18,103.20
2005,Japan,JPN,False,2019-05-01,Solar,False,59.57,469.70
4009,India,IND,False,2022-03-01,Solar,False,70.20,49.54


In [257]:
df_generation_yearly.sample(3)

,entity,entity_code,is_aggregate_entity,date,series,is_aggregate_series,generation_twh,share_of_generation_pct
38154,Lebanon,LBN,False,2017,Solar,False,0.06,0.28
26595,Kiribati,KIR,False,2012,Bioenergy,False,0.00,0.00
24390,Iraq,IRQ,False,2011,Other renewables,False,0.00,0.00


## Dataset Overview

In [258]:
# remove is_aggregate_series column since it is constant
if "is_aggregate_series" in df_generation_yearly.columns:
    df_generation_yearly = df_generation_yearly.drop(columns="is_aggregate_series")
    df_capacity_monthly = df_capacity_monthly.drop(columns="is_aggregate_series")

# rename date to year since only year data is there
# rename series to technology to be more descriptive
df_generation_yearly = df_generation_yearly\
    .rename(columns={"date":"year",
                     "series":"technology"})

# rename series to technology to be more descriptive
# rename date to month since only month data is there
df_capacity_monthly = df_capacity_monthly\
    .rename(columns={"series":"technology",
                     "date":"month"})
# add year column
df_capacity_monthly['year'] = pd.to_datetime(df_capacity_monthly['month']).dt.year

### Electricity Generation Yearly

In [259]:
df_generation_yearly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52181 entries, 0 to 52180
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   entity                   52181 non-null  object 
 1   entity_code              49231 non-null  object 
 2   is_aggregate_entity      52181 non-null  bool   
 3   year                     52181 non-null  object 
 4   technology               52181 non-null  object 
 5   generation_twh           52181 non-null  float64
 6   share_of_generation_pct  52181 non-null  float64
dtypes: bool(1), float64(2), object(4)
memory usage: 2.4+ MB


In [260]:
df_generation_yearly.isnull().sum()

entity                        0
entity_code                2950
is_aggregate_entity           0
year                          0
technology                    0
generation_twh                0
share_of_generation_pct       0
dtype: int64

In [261]:
df_generation_yearly[df_generation_yearly['is_aggregate_entity']==True]['entity_code'].isnull().sum()

np.int64(2950)

No missing data. The only nulls are entity codes which are not defined for those aggregated regions such as EU, Asia, etc.

#### Split the dataset into country/economy and aggregated region

In [262]:
# split dataset into aggregated regions and individual country/economy dataset
# aggregated region dataset
df_generation_region = df_generation_yearly[df_generation_yearly['is_aggregate_entity'] == True].copy()
df_generation_country = df_generation_yearly[df_generation_yearly['is_aggregate_entity'] == False].copy()

# drop is_aggregated_entity column since it is constant in each dataset after splitting
df_generation_region = df_generation_region.drop(columns="is_aggregate_entity")
df_generation_country = df_generation_country.drop(columns="is_aggregate_entity")

print(df_generation_region.shape, df_generation_country.shape)

(2950, 6) (49231, 6)


In [263]:
print("Num Countries/Economies: ", df_generation_country['entity'].nunique())
print("Num Aggregated Regions: ", df_generation_region['entity'].nunique())

Num Countries/Economies:  210
Num Aggregated Regions:  13


In [264]:
df_generation_region['entity'].unique()

array(['World', 'Middle East', 'Latin America and Caribbean',
       'North America', 'Oceania', 'OECD', 'Africa', 'ASEAN', 'Asia',
       'G20', 'G7', 'EU', 'Europe'], dtype=object)

Our dataset covers 13 aggregated regions and 210 countries.

##### Country/Economy-wise Variables

In [265]:
df_generation_country.info() # no missing data

<class 'pandas.core.frame.DataFrame'>
Index: 49231 entries, 0 to 52170
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   entity                   49231 non-null  object 
 1   entity_code              49231 non-null  object 
 2   year                     49231 non-null  object 
 3   technology               49231 non-null  object 
 4   generation_twh           49231 non-null  float64
 5   share_of_generation_pct  49231 non-null  float64
dtypes: float64(2), object(4)
memory usage: 2.6+ MB


In [266]:
# do we have full years for each countries?
df_generation_country.groupby("entity").agg(
    year_count = pd.NamedAgg(column = "year", aggfunc=pd.Series.nunique),
    min_year = pd.NamedAgg(column = "year", aggfunc=pd.Series.min),
    max_year = pd.NamedAgg(column = "year", aggfunc=pd.Series.max)
).sort_values(by="year_count")

,year_count,min_year,max_year
entity,,,
South Sudan,12,2012,2023
Montenegro,20,2005,2024
Cabo Verde,23,2000,2022
Bahamas,23,2000,2022
Falkland Islands (Malvinas),23,2000,2022
...,...,...,...
United Kingdom,25,2000,2024
United Arab Emirates,25,2000,2024
Türkiye,25,2000,2024


Almost all countries have generation data from the year 2000 except for South Sudan.

In [267]:
df_generation_country['technology'].unique()

array(['Bioenergy', 'Coal', 'Gas', 'Hydro', 'Net imports', 'Nuclear',
       'Other fossil', 'Other renewables', 'Solar', 'Wind'], dtype=object)

In [268]:
print(df_generation_country['technology'].nunique())
df_generation_country['technology'].value_counts().sort_values()

10


technology
Other renewables    4459
Nuclear             4712
Wind                4905
Gas                 4948
Hydro               4981
Coal                4993
Bioenergy           5032
Solar               5054
Net imports         5073
Other fossil        5074
Name: count, dtype: int64

Dataset have 9 different categories of energy generation excluding imports.

In [269]:
# when is the earlies time recorded for each energy category
df_generation_country.groupby("technology").agg(
    min_year = pd.NamedAgg(column="year", aggfunc="min")
)

,min_year
technology,
Bioenergy,2000
Coal,2000
Gas,2000
Hydro,2000
Net imports,2000
Nuclear,2000
Other fossil,2000
Other renewables,2000
Solar,2000


##### Aggregated regions

In [270]:
df_generation_region.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2950 entries, 78 to 52180
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   entity                   2950 non-null   object 
 1   entity_code              0 non-null      object 
 2   year                     2950 non-null   object 
 3   technology               2950 non-null   object 
 4   generation_twh           2950 non-null   float64
 5   share_of_generation_pct  2950 non-null   float64
dtypes: float64(2), object(4)
memory usage: 161.3+ KB


In [271]:
df_generation_region.entity.value_counts()

entity
EU                             250
Middle East                    225
Latin America and Caribbean    225
North America                  225
World                          225
Oceania                        225
OECD                           225
ASEAN                          225
Africa                         225
Asia                           225
G20                            225
G7                             225
Europe                         225
Name: count, dtype: int64

In [272]:
# do we have full years for each regions
df_generation_region.groupby("entity").agg(
    year_count = pd.NamedAgg(column = "year", aggfunc=pd.Series.nunique),
    min_year = pd.NamedAgg(column = "year", aggfunc=pd.Series.min),
    max_year = pd.NamedAgg(column = "year", aggfunc=pd.Series.max)
).sort_values(by="year_count")

,year_count,min_year,max_year
entity,,,
ASEAN,25,2000,2024
Africa,25,2000,2024
Asia,25,2000,2024
EU,25,2000,2024
Europe,25,2000,2024
G20,25,2000,2024
G7,25,2000,2024
Latin America and Caribbean,25,2000,2024
Middle East,25,2000,2024


We have electricity generation data from 2000 till 2024 for all aggregated regions.

In [273]:
df_generation_region['technology'].value_counts().sort_values()

technology
Net imports          25
Bioenergy           325
Gas                 325
Coal                325
Nuclear             325
Other fossil        325
Other renewables    325
Hydro               325
Solar               325
Wind                325
Name: count, dtype: int64

In [274]:
# net import data is only available for EU region
df_generation_region[df_generation_region['technology']=="Net imports"]['entity'].value_counts()

entity
EU    25
Name: count, dtype: int64

In [275]:
# when is the earlies time recorded for each energy category
df_generation_region.groupby("technology").agg(
    min_year = pd.NamedAgg(column="year", aggfunc="min"),
    max_year = pd.NamedAgg(column="year", aggfunc="max"),
)

,min_year,max_year
technology,,
Bioenergy,2000,2024
Coal,2000,2024
Gas,2000,2024
Hydro,2000,2024
Net imports,2000,2024
Nuclear,2000,2024
Other fossil,2000,2024
Other renewables,2000,2024
Solar,2000,2024


In [276]:
# filter for world electricity generation trends only
df_generation_world = df_generation_region[df_generation_region['entity']=='World'].copy()
df_generation_world.sample(4)

,entity,entity_code,year,technology,generation_twh,share_of_generation_pct
16215,World,None,2007,Other renewables,52.81,0.27
42111,World,None,2019,Other fossil,854.85,3.18
34069,World,None,2015,Other fossil,1133.36,4.72
25552,World,None,2011,Nuclear,2610.34,11.88


In [277]:
total_yearly_gen = df_generation_world.groupby(['technology', 'year']).agg(
    total_yearly_gen = pd.NamedAgg(column="generation_twh", aggfunc="sum") 
).reset_index()

# Plot Electricity Generation Over Time based on sum of all countries yearly
fig = px.area(total_yearly_gen, x='year', y='total_yearly_gen', color='technology',
                  title='Global Share of Renewable vs Non-Renewable Energy Generation Over Time')
fig.update_layout(xaxis_title='Year', yaxis_title='Electricity Generation (tWh)')
fig.show() # for interactive image
# save_static_img(fig, DATA_DIR / "eda_interim" / "figures" / "global_generation_share_over_time.png") # since github doesn't render px plots

In [278]:
# compare with total sum of coutntries
total_yearly_gen = df_generation_country.groupby(['technology', 'year']).agg(
    total_yearly_gen = pd.NamedAgg(column="generation_twh", aggfunc="sum") 
).reset_index()

# Plot Electricity Generation Over Time based on sum of all countries yearly
fig = px.area(total_yearly_gen, x='year', y='total_yearly_gen', color='technology',
                  title='Global Share of Renewable vs Non-Renewable Energy Generation Over Time')
fig.update_layout(xaxis_title='Year', yaxis_title='Electricity Generation (tWh)')
fig.show() # for interactive image
# save_static_img(fig, DATA_DIR / "eda_interim" / "figures" / "global_generation_share_over_time.png") # since github doesn't render px plots

df_generation_yearly
- Data sourced from Ember API, an open platform for worldwide energy data.
- contains energy generation data for individual Countries/Economies and aggregated economies as well like (EU, OECD, Asia, etc)
- split the dataset by country/economy vs aggregated region: 
	> per Country/Economy dataset: 49,231 rows, covering 210 unique economies. <br>
	> Aggregated Region dataset: 2,950 rows, covering 13 unique regions.

- energy generation is measured in Twh (terawatt hours)
- for most countries, the data covers from 2000 to 2024 except for few countries.
- covers 9 energy technology types excluding imports
	> 'Bioenergy', 'Coal', 'Gas', 'Hydro', 'Net imports', 'Nuclear', 'Other fossil', 'Other renewables', 'Solar', 'Wind'


### Electricity Capacity Yearly

In [279]:
df_capacity_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6679 entries, 0 to 6678
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   entity                 6679 non-null   object 
 1   entity_code            6679 non-null   object 
 2   is_aggregate_entity    6679 non-null   bool   
 3   month                  6679 non-null   object 
 4   technology             6679 non-null   object 
 5   capacity_gw            6679 non-null   float64
 6   capacity_w_per_capita  6679 non-null   float64
 7   year                   6679 non-null   int32  
dtypes: bool(1), float64(2), int32(1), object(4)
memory usage: 345.8+ KB


In [280]:
df_capacity_monthly.isnull().sum()

entity                   0
entity_code              0
is_aggregate_entity      0
month                    0
technology               0
capacity_gw              0
capacity_w_per_capita    0
year                     0
dtype: int64

No missing data.

In [281]:
df_capacity_monthly['year'].value_counts()

year
2023    744
2024    744
2022    732
2021    720
2020    684
2019    670
2018    612
2025    595
2017    590
2016    588
Name: count, dtype: int64

In [282]:
df_capacity_monthly['is_aggregate_entity'].value_counts()

is_aggregate_entity
False    6679
Name: count, dtype: int64

Unlike the generation data, capacity data doesn't have aggregated region data.

In [ ]:
# drop the is_aggregate_entity col since it is constant
# and rename the df_capacity_monthly to df_capacity_country for consistency
df_capacity_country = df_capacity_monthly.drop(columns="is_aggregate_entity")

In [284]:
# how many countries/economies
df_capacity_country['entity'].nunique()

25

In [285]:
# range of time period covered in dataset
df_capacity_monthly['month'].min(), df_capacity_monthly['month'].max() 

('2016-01-01', '2025-11-01')

We have capacity data for only 25 countries which is significantly smaller set compared to the generation data covering 210 countries.

In [286]:
# add a year column for easier aggregation
df_capacity_country['year'] = pd.to_datetime(df_capacity_country['month']).dt.year

In [287]:
# do we have full years for each countries?
df_capacity_country.groupby("entity").agg(
    year_count = pd.NamedAgg(column = "year", aggfunc=pd.Series.nunique),
    min_year = pd.NamedAgg(column = "year", aggfunc=pd.Series.min),
    max_year = pd.NamedAgg(column = "year", aggfunc=pd.Series.max)
).sort_values(by="year_count")

,year_count,min_year,max_year
entity,,,
South Africa,4,2022,2025
Italy,5,2021,2025
India,7,2019,2025
Chile,7,2019,2025
South Korea,9,2017,2025
China,10,2016,2025
Brazil,10,2016,2025
Argentina,10,2016,2025
Finland,10,2016,2025


In [288]:
df_capacity_country['technology'].value_counts().sort_values()

technology
Wind unspecified     215
Wind offshore       1351
Wind onshore        2487
Solar               2626
Name: count, dtype: int64

In [289]:
# when is the earlies time recorded for each energy category
df_capacity_country.groupby("technology").agg(
    min_year = pd.NamedAgg(column="year", aggfunc="min")
)

,min_year
technology,
Solar,2016
Wind offshore,2016
Wind onshore,2016
Wind unspecified,2016


Types of energy technology covered in this dataset is also incomplete compare to more comprehensive set in generation data.

In [290]:
df_capacity_country.sample(3)

,entity,entity_code,month,technology,capacity_gw,capacity_w_per_capita,year
2646,Chile,CHL,2020-04-01,Solar,3.02,156.81,2020
1081,Netherlands,NLD,2017-11-01,Solar,2.61,150.99,2017
1404,Türkiye,TUR,2018-05-01,Solar,4.89,58.46,2018


In [291]:
# compare with total sum of coutntries
total_yearly_capacity = df_capacity_country.groupby(['technology', 'year']).agg(
    total_yearly_capacity = pd.NamedAgg(column="capacity_gw", aggfunc="sum") 
).reset_index()

# Plot Electricity Generation Over Time based on sum of all countries yearly
fig = px.area(total_yearly_capacity, x='year', y='total_yearly_capacity', color='technology',
                  title='Global Share of Renewable vs Non-Renewable Energy Capacity Over Time')
fig.update_layout(xaxis_title='Year', yaxis_title='Electricity Capacity (GW)')
fig.show() # for interactive image
# save_static_img(fig, DATA_DIR / "eda_interim" / "figures" / "global_generation_share_over_time.png") # since github doesn't render px plots

df_capacity_monthly
- Data sourced from Ember API, an open platform for worldwide energy data.
- contains energy capacity installed data for individual Countries/Economies only
	> 6679 rows covers 25 unique economies
- energy capacity is measured in Gw (gigawatts)
- for most countries, the data covers from 2016 to 2025
- covers 2 energy technology types only
	> wind (unspecified, offshore, onshore) and solar


Compared to the energy generation dataset, this capacity data feels significantly less complete. If we need better coverage for energy technology type and time period to compare capacity vs generation, we'll need to get more data elsewhere. One id a

## Yearly Full Release Dataset 

As we have seen from above overview, the API data is not complete especially for capacity data. Ember Energy website also have a yearly electricity dataset that be downloaded directly.
We will also check that dataset to see if we can have the data we want in there. 

In [292]:
file_url = "https://storage.googleapis.com/emb-prod-bkt-publicdata/public-downloads/yearly_full_release_long_format.csv"
yearly_full_release = download_from_url(file_url, DATA_DIR / 'eda_interim')

2026-01-04 23:15:54 | INFO | Downloading data from https://storage.googleapis.com/emb-prod-bkt-publicdata/public-downloads/yearly_full_release_long_format.csv to /home/zephyr/workspace/Global_Energy_Trends/data/eda_interim/yearly_full_release_long_format.csv...
2026-01-04 23:15:57 | INFO | Download complete.


In [293]:
full_release_df = pd.read_csv(yearly_full_release)
full_release_df.sample(3)

,Area,ISO 3 code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,ASEAN,Category,Subcategory,Variable,Unit,Value,YoY absolute change,YoY % change
112819,Finland,FIN,2022,Country or economy,Europe,Europe,1.0,1.0,0.0,0.0,0.0,Electricity generation,Aggregate fuel,Clean,TWh,64.00,2.22,3.59
36047,Benin,BEN,2013,Country or economy,Africa,Africa,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Other Fossil,TWh,0.09,0.04,80.00
350992,World,NaN,2005,Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Capacity,Fuel,Other Renewables,GW,13.38,0.64,5.02


In [294]:
print("Area Types: ", full_release_df['Area type'].unique())
print("Subcategories: ", full_release_df['Subcategory'].unique())
print("Unaggregated Fuel Types: ", full_release_df[full_release_df['Subcategory']=="Fuel"]['Variable'].unique())
print(full_release_df["Category"].unique())

Area Types:  ['Country or economy' 'Region']
Subcategories:  ['Aggregate fuel' 'Fuel' 'Demand' 'Demand per capita' 'Total'
 'Electricity imports' 'CO2 intensity']
Unaggregated Fuel Types:  ['Bioenergy' 'Coal' 'Gas' 'Hydro' 'Nuclear' 'Other Fossil'
 'Other Renewables' 'Solar' 'Wind']
['Capacity' 'Electricity demand' 'Electricity generation'
 'Electricity imports' 'Power sector emissions']


The dataset looks comprehensive and include both generation, capacity and other categories as well such as demand, import and emissions.

We'll first compare the completeness with dataset we got from API. If this one is better, we'll use it.

In [295]:
# rename columns for easier comparison
full_release_df = full_release_df.rename(columns={
                    "Area":"entity",
                    "Variable":"technology"
                })

In [296]:
# split dataset into aggregated regions and individual country/economy dataset
# for generation and capacity
# filtering based on Unit since this data also have % units and we only want Twh
df_generation_country = full_release_df.query('`Area type` == "Country or economy" and Subcategory == "Fuel" and Category == "Electricity generation" and Unit == "TWh"')
df_generation_region = full_release_df.query('`Area type` == "Region" and Subcategory == "Fuel" and Category == "Electricity generation" and Unit == "TWh"')

df_capacity_country = full_release_df.query('`Area type` == "Country or economy" and Subcategory == "Fuel" and Category == "Capacity" and Unit == "GW"')
df_capacity_region = full_release_df.query('`Area type` == "Region" and Subcategory == "Fuel" and Category == "Capacity" and Unit == "GW"')

#### Yearly Electricity Generation 
Compare with the dataset from API

In [297]:
df_generation_country.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45076 entries, 38 to 357032
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   entity               45076 non-null  object 
 1   ISO 3 code           45076 non-null  object 
 2   Year                 45076 non-null  int64  
 3   Area type            45076 non-null  object 
 4   Continent            45076 non-null  object 
 5   Ember region         45076 non-null  object 
 6   EU                   45076 non-null  float64
 7   OECD                 45076 non-null  float64
 8   G20                  45076 non-null  float64
 9   G7                   45076 non-null  float64
 10  ASEAN                45076 non-null  float64
 11  Category             45076 non-null  object 
 12  Subcategory          45076 non-null  object 
 13  technology           45076 non-null  object 
 14  Unit                 45076 non-null  object 
 15  Value                45076 non-null  fl

In [298]:
df_generation_country['entity'].nunique() # 215

215

215 economies listed in the generation data whereas API data had 210 countries.

In [299]:
# do we have full years for each countries?
df_generation_country.groupby("entity").agg(
    year_count = pd.NamedAgg(column = "Year", aggfunc=pd.Series.nunique),
    min_year = pd.NamedAgg(column = "Year", aggfunc=pd.Series.min),
    max_year = pd.NamedAgg(column = "Year", aggfunc=pd.Series.max)
).sort_values(by="year_count")

,year_count,min_year,max_year
entity,,,
Western Sahara,10,2000,2009
South Sudan,12,2012,2023
Montenegro,20,2005,2024
Timor-Leste,21,2003,2023
Faroe Islands (the),23,2000,2022
...,...,...,...
United Kingdom,25,2000,2024
United States of America,25,2000,2024
Tunisia,25,2000,2024


Range of data available is almost similar to the API data which is 2000 to 2024 for most economies in the list.

In [300]:
print(df_generation_country['technology'].nunique())
df_generation_country['technology'].unique()

9


array(['Bioenergy', 'Coal', 'Gas', 'Hydro', 'Nuclear', 'Other Fossil',
       'Other Renewables', 'Solar', 'Wind'], dtype=object)

9 energy technology type covered. Same as the API data.

In [301]:
print(df_generation_country['technology'].nunique())
df_generation_country['technology'].value_counts().sort_values()

9


technology
Other Renewables    4561
Nuclear             4814
Wind                5007
Gas                 5050
Hydro               5083
Coal                5095
Bioenergy           5134
Solar               5156
Other Fossil        5176
Name: count, dtype: int64

We have 5 more countries in this dataset than the one from API, which is probably why we have a few more data points here.

In [302]:
# when is the earlies time recorded for each energy category
df_generation_country.groupby("technology").agg(
    min_year = pd.NamedAgg(column="Year", aggfunc="min")
)

,min_year
technology,
Bioenergy,2000
Coal,2000
Gas,2000
Hydro,2000
Nuclear,2000
Other Fossil,2000
Other Renewables,2000
Solar,2000
Wind,2000


**Aggregated Regions**

In [303]:
df_generation_region.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2925 entries, 1622 to 352280
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   entity               2925 non-null   object 
 1   ISO 3 code           0 non-null      object 
 2   Year                 2925 non-null   int64  
 3   Area type            2925 non-null   object 
 4   Continent            0 non-null      object 
 5   Ember region         0 non-null      object 
 6   EU                   0 non-null      float64
 7   OECD                 0 non-null      float64
 8   G20                  0 non-null      float64
 9   G7                   0 non-null      float64
 10  ASEAN                0 non-null      float64
 11  Category             2925 non-null   object 
 12  Subcategory          2925 non-null   object 
 13  technology           2925 non-null   object 
 14  Unit                 2925 non-null   object 
 15  Value                2925 non-null   f

We have 2950 non-null in API data which is 25 more rows than here.  

In [304]:
df_generation_region.entity.value_counts()

entity
Africa                         225
ASEAN                          225
Asia                           225
EU                             225
Europe                         225
G20                            225
G7                             225
Latin America and Caribbean    225
Middle East                    225
North America                  225
Oceania                        225
OECD                           225
World                          225
Name: count, dtype: int64

In [305]:
# do we have full years for each regions
df_generation_region.groupby("entity").agg(
    year_count = pd.NamedAgg(column = "Year", aggfunc=pd.Series.nunique),
    min_year = pd.NamedAgg(column = "Year", aggfunc=pd.Series.min),
    max_year = pd.NamedAgg(column = "Year", aggfunc=pd.Series.max)
).sort_values(by="year_count")

,year_count,min_year,max_year
entity,,,
ASEAN,25,2000,2024
Africa,25,2000,2024
Asia,25,2000,2024
EU,25,2000,2024
Europe,25,2000,2024
G20,25,2000,2024
G7,25,2000,2024
Latin America and Caribbean,25,2000,2024
Middle East,25,2000,2024


Coverage is same as API data

In [306]:
df_generation_region['technology'].value_counts().sort_values()

technology
Bioenergy           325
Coal                325
Gas                 325
Hydro               325
Nuclear             325
Other Fossil        325
Other Renewables    325
Solar               325
Wind                325
Name: count, dtype: int64

Same technology types but API data also have net imports category. But it is only for EU.

In [307]:
# filter for world electricity generation trends only
df_generation_world = df_generation_region[df_generation_region['entity']=='World'].copy()
df_generation_world.sample(3)

,entity,ISO 3 code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,ASEAN,Category,Subcategory,technology,Unit,Value,YoY absolute change,YoY % change
352277,World,NaN,2024,Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electricity generation,Fuel,Other Fossil,TWh,869.74,-2.34,-0.27
350894,World,NaN,2003,Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electricity generation,Fuel,Wind,TWh,63.19,10.97,21.01
351686,World,NaN,2015,Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electricity generation,Fuel,Wind,TWh,829.52,123.50,17.49


In [308]:
total_yearly_gen = df_generation_world.groupby(['technology', 'Year']).agg(
    total_yearly_gen = pd.NamedAgg(column="Value", aggfunc="sum") 
).reset_index()

# Plot Electricity Generation Over Time based on sum of all countries yearly
fig = px.area(total_yearly_gen, x='Year', y='total_yearly_gen', color='technology',
                  title='Global Share of Renewable vs Non-Renewable Energy Generation Over Time')
fig.update_layout(xaxis_title='Year', yaxis_title='Electricity Generation (tWh)')
fig.show() # for interactive image
# save_static_img(fig, DATA_DIR / "eda_interim" / "figures" / "global_generation_share_over_time.png") # since github doesn't render px plots

Based on the findings above, both API data and data here are identical except for EU net imports. This data covers also a few more economies than the API data.

#### Yearly Electricity Capacity Installed 
Compare with the dataset from API

In [310]:
df_capacity_country["Year"].value_counts().sort_index()

Year
2000    1827
2001    1826
2002    1828
2003    1839
2004    1847
2005    1857
2006    1859
2007    1858
2008    1856
2009    1858
2010    1852
2011    1856
2012    1866
2013    1867
2014    1873
2015    1874
2016    1874
2017    1875
2018    1875
2019    1876
2020    1876
2021    1875
2022    1877
2023    1526
2024     779
Name: count, dtype: int64

The capacity data here begins from 2000 whereas the API data starts from 2016 only.

In [311]:
# how many countries/economies
df_capacity_country['entity'].nunique()

215

This data covers 215 entities, and the API data covers only 25 countries.

In [312]:
# do we have full years for each countries?
df_capacity_country.groupby("entity").agg(
    year_count = pd.NamedAgg(column = "Year", aggfunc=pd.Series.nunique),
    min_year = pd.NamedAgg(column = "Year", aggfunc=pd.Series.min),
    max_year = pd.NamedAgg(column = "Year", aggfunc=pd.Series.max)
).sort_values(by="year_count")

,year_count,min_year,max_year
entity,,,
Western Sahara,10,2000,2009
South Sudan,12,2012,2023
Montenegro,20,2005,2024
Timor-Leste,21,2003,2023
Faroe Islands (the),23,2000,2022
...,...,...,...
United Kingdom,25,2000,2024
United States of America,25,2000,2024
Tunisia,25,2000,2024


In [313]:
df_capacity_country['technology'].value_counts().sort_values()

technology
Other Renewables    4561
Nuclear             4814
Wind                5007
Gas                 5050
Hydro               5083
Coal                5095
Bioenergy           5134
Solar               5156
Other Fossil        5176
Name: count, dtype: int64

In [315]:
# when is the earlies time recorded for each energy category
df_capacity_country.groupby("technology").agg(
    min_year = pd.NamedAgg(column="Year", aggfunc="min")
)

,min_year
technology,
Bioenergy,2000
Coal,2000
Gas,2000
Hydro,2000
Nuclear,2000
Other Fossil,2000
Other Renewables,2000
Solar,2000
Wind,2000


## Final Decision on Dataset Selection
After comparing the datasets from the Ember API and their Yearly Full Release CSV, I have decided to use the Yearly Full Release Dataset as the primary source for this analysis. Although I prefer using Ember's API for our pipeline integration, we chose the CSV download for the below reasons.

**Electricity Generation Data**: <br>
Both datasets are nearly identical in coverage (215 countries/economies vs. 210 from API, 2000-2024 time range, 9 technology types). The Full Release includes a few more economies and is slightly more complete.

**Electricity Capacity Data**: <br>
The Full Release is significantly more comprehensive, covering 215 entities from 2000 onwards with broader technology types (e.g., including more renewables), compared to the API's limited 25 countries starting from 2016.

**Overall Completeness**: 
The Full Release provides a more holistic view of global energy trends, including additional categories like demand, imports, and emissions, which could be useful for future extensions. It also aligns better with the generation data in terms of entity coverage and time periods.

**Data Integrity**: 
No major discrepancies were found; the Full Release appears to be a superset of the API data, making it the superior choice for a complete EDA.  This dataset will be used to proceed with further analysis, transformations, and visualizations in subsequent steps.